In [1]:
#!/usr/bin/python
# -*- coding: utf-8 -*-
import warnings
warnings.filterwarnings('ignore')

In [2]:
import tensorflow
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input, Dense, Activation, Conv2D, MaxPooling2D, Dropout, GlobalAveragePooling2D, InputLayer, Flatten, BatchNormalization, Reshape, Lambda

from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.optimizers import RMSprop, Adam
from tensorflow.keras.preprocessing.image import load_img, ImageDataGenerator
from tensorflow.keras.utils import multi_gpu_model
from IPython.display import Image 

import keras_metrics
import pandas
import ast
import numpy as np
import matplotlib.patches as patches 
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from PIL import Image, ImageDraw

Using TensorFlow backend.


In [3]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

In [4]:
import keras.backend as K
import numpy as np
import random

def yolf_loss(y_true, y_pred):
    b_p_pred = y_pred[..., :0]
    b_xy_pred = y_pred[..., 1:3]
    b_wh_pred = y_pred[..., 3:5]
    
    b_p = y_true[..., 0]
    b_xy = y_true[..., 1:3]
    b_wh = y_true[..., 3:5]
    #print(b_xy_pred.get_shape(),b_xy.get_shape())
    #print(b_wh_pred.get_shape(),b_wh.get_shape())
    #indicator_coord = K.expand_dims(y_true[..., 3], axis=-1) * 1.0
    loss_p = K.sum(K.square(b_p - b_p_pred), axis=-1)
    loss_xy = K.sum(b_p * K.square(b_xy - b_xy_pred), axis=-1)# * indicator_coord)#, axis=[1,2,3,4])
    loss_wh = K.sum(b_p * K.square(K.sqrt(b_wh) - K.sqrt(b_wh_pred)), axis=-1)# * indicator_coord)#, axis=[1,2,3,4])
    return (loss_p + loss_wh + loss_xy)/3

In [5]:
train_set = ".\\dataset\\NUOVO\\train\\"
validation_set = ".\\dataset\\NUOVO\\val\\"
test_set = ".\\dataset\\NUOVO\\test\\"

In [6]:
train_df = pandas.read_csv(".\\dataset\\NUOVO\\train\\output_segmentation.csv")
valid_df = pandas.read_csv(".\\dataset\\NUOVO\\val\\output_segmentation.csv")
test_df = pandas.read_csv(".\\dataset\\NUOVO\\test\\output_segmentation.csv")

train_df['label'] = train_df['label'].apply(lambda x: ast.literal_eval(x))
valid_df['label'] = valid_df['label'].apply(lambda x: ast.literal_eval(x))
test_df['label'] = test_df['label'].apply(lambda x: ast.literal_eval(x))

IMG_SIZE = 224
BLOCKS_NUM = 19
BLOCK_SIZE = IMG_SIZE / BLOCKS_NUM

path_train_image = ".\\dataset\\NUOVO\\train"
count = 0 
for index,row in train_df.iterrows(): 
    im = Image.open(".\\dataset\\NUOVO\\train\\" + row["image_id"], 'r')
    draw = ImageDraw.Draw(im)
    for i in range(BLOCKS_NUM):
        for j in range(BLOCKS_NUM):

            #print(ast.literal_eval(row["label"])[(i*19)+j])
            
            if(row["label"][(i*19)+j][0] == 1):
                draw.rectangle(((i * BLOCK_SIZE, j * BLOCK_SIZE), (i * BLOCK_SIZE + BLOCK_SIZE, j * BLOCK_SIZE + BLOCK_SIZE)), outline="Green")
            else:
                draw.rectangle(((i * BLOCK_SIZE, j * BLOCK_SIZE), (i * BLOCK_SIZE + BLOCK_SIZE, j * BLOCK_SIZE + BLOCK_SIZE)), outline="Black")
    count += 1
    im.show()
    if(count == 1):
        break
    

train_df['label'] = train_df['label'].apply(lambda x: np.asarray(x).flatten())
valid_df['label'] = valid_df['label'].apply(lambda x: np.asarray(x).flatten())
test_df['label'] = test_df['label'].apply(lambda x: np.asarray(x).flatten())

for el in range(valid_df.label[0].shape[0]):
    valid_df[str(el)] = 0

valid_df.head(3)

len(valid_df.label[0])

count = 0
for index, row in valid_df.iterrows():
    count = count +1
    if count%1000==0:
        print("Done", count, "rows")
    for el in range(valid_df.label[0].shape[0]):
        valid_df[str(el)][index] = valid_df['label'][index][el]

for el in range(valid_df.label[0].shape[0]):
    print(valid_df[str(el)][0])

valid_df.head(3)

In [8]:
valid_df.columns[1:]

Index(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9',
       ...
       '235', '236', '237', '238', '239', '240', '241', '242', '243', '244'],
      dtype='object', length=245)

In [16]:
BATCH_SIZE = 32
IMG_SIZE = 224
train_datagen = ImageDataGenerator(
    rescale=1./255)

train_generator = train_datagen.flow_from_dataframe(
        dataframe=train_df,
        directory=train_set,
        x_col="image_id",
        y_col=train_df.columns[1:],
        class_mode="raw",
        shuffle=True,
        target_size=(IMG_SIZE, IMG_SIZE),
        batch_size=BATCH_SIZE)

valid_generator = train_datagen.flow_from_dataframe(
        dataframe=valid_df,
        directory=validation_set,
        x_col="image_id",
        y_col=valid_df.columns[1:],
        class_mode="raw",
        shuffle=True,
        target_size=(IMG_SIZE, IMG_SIZE),
        batch_size=BATCH_SIZE)

Found 164105 validated image filenames.
Found 18234 validated image filenames.


train_generator.reset()

for el in train_generator:
    images = el[0]
    label = el[1]
    for l in label:
        print(l)

In [17]:
step_size_train = train_generator.samples/train_generator.batch_size
step_size_valid = valid_generator.samples/valid_generator.batch_size

true_boxes  = Input(shape=(1, 1, 1, 1, 1))
#input_image = Input(shape=(224, 224, 3))
input_image = InputLayer(input_shape=(224,224,3))

model = Conv2D(filters=32, kernel_size=(3,3), padding="same", activation="relu")(input_image)
model = BatchNormalization()(model)
model = MaxPooling2D(pool_size=(2,2), strides=2)(model)

model = Conv2D(filters=64, kernel_size=(3,3), padding="same", activation="relu")(model)
model = BatchNormalization()(model)
model = MaxPooling2D(pool_size=(2,2), strides=2)(model)

model = Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu")(model)
model = Conv2D(filters=64, kernel_size=(1,1), activation="relu")(model)
model = Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu")(model)
model = BatchNormalization()(model)
model = MaxPooling2D(pool_size=(2,2), strides=2)(model)

model = Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu")(model)
model = Conv2D(filters=128, kernel_size=(1,1), activation="relu")(model)
model = Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu")(model)
model = BatchNormalization()(model)
model = MaxPooling2D(pool_size=(2,2), strides=2)(model)

model = Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu")(model)
model = Conv2D(filters=256, kernel_size=(1,1), activation="relu")(model)
model = Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu")(model)
model = Conv2D(filters=256, kernel_size=(1,1), activation="relu")(model)
model = Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu")(model)
model = BatchNormalization()(model)
model = MaxPooling2D(pool_size=(2,2), strides=2)(model)
          
model = Conv2D(filters=1024, kernel_size=(3,3), padding="same", activation="relu")(model)
model = Conv2D(filters=512, kernel_size=(1,1), activation="relu")(model)
model = Conv2D(filters=1024, kernel_size=(3,3), padding="same", activation="relu")(model)
model = Conv2D(filters=512, kernel_size=(1,1), activation="relu")(model)
model = Conv2D(filters=1024, kernel_size=(3,3), padding="same", activation="relu")(model)

model = Conv2D(filters=5, kernel_size=(1,1), activation="relu")(model)

model = Reshape((7, 7, 5))(model)

#output = Lambda(lambda args: args[0])([model, true_boxes])

model = Sequential(input_image, output)


#model = Conv2D(filters=4, kernel_size=(1,1)))
#model = GlobalAveragePooling2D())
#model = BatchNormalization())

In [18]:
model = Sequential()
model.add(InputLayer(input_shape=(224,224,3)))
model.add(Conv2D(filters=32, kernel_size=(3,3), padding="same", activation="relu"))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2), strides=2))

model.add(Conv2D(filters=64, kernel_size=(3,3), padding="same", activation="relu"))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2), strides=2))

model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=64, kernel_size=(1,1), activation="relu"))
model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2), strides=2))

model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=128, kernel_size=(1,1), activation="relu"))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2), strides=2))

model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=256, kernel_size=(1,1), activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=256, kernel_size=(1,1), activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2), strides=2))
          
model.add(Conv2D(filters=1024, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(1,1), activation="relu"))
model.add(Conv2D(filters=1024, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(1,1), activation="relu"))
model.add(Conv2D(filters=1024, kernel_size=(3,3), padding="same", activation="relu"))

model.add(Conv2D(filters=5, kernel_size=(1,1), activation="relu"))

#model.add(Reshape((245, 1)))

model.add(Flatten())

#output = Lambda(lambda args: args[0])([model, true_boxes])

#model = Sequential(input_image, output)

#model.add(Conv2D(filters=4, kernel_size=(1,1)))
#model.add(GlobalAveragePooling2D())
#model.add(BatchNormalization())

In [19]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_19 (Conv2D)           (None, 224, 224, 32)      896       
_________________________________________________________________
batch_normalization_5 (Batch (None, 224, 224, 32)      128       
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 112, 112, 32)      0         
_________________________________________________________________
conv2d_20 (Conv2D)           (None, 112, 112, 64)      18496     
_________________________________________________________________
batch_normalization_6 (Batch (None, 112, 112, 64)      256       
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 56, 56, 64)        0         
_________________________________________________________________
conv2d_21 (Conv2D)           (None, 56, 56, 128)      

model = multi_gpu_model(model, gpus=2) #parallelize model

In [20]:
model.compile(optimizer=Adam(lr = 1e-4), loss="mse", metrics=["mse"])

In [21]:
model.evaluate_generator(valid_generator, steps=step_size_valid, verbose = 1)

570/569 [==============================] - 31s 55ms/step - loss: 1980.6954 - mse: 1980.7390


[1981.3460120430925, 1980.739]

In [23]:
earlyStopping = EarlyStopping(monitor='val_loss', patience=50, verbose=1, mode='min')
mcp_save = ModelCheckpoint('yolf.h5', save_best_only=True, monitor='val_loss', mode='min')
reduce_lr_loss = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=10, verbose=1, mode='min')

history = model.fit_generator(generator=train_generator, epochs=10, steps_per_epoch=step_size_train, validation_data=valid_generator, validation_steps=step_size_valid, verbose=1, callbacks=[mcp_save, earlyStopping, reduce_lr_loss])

Epoch 1/10
5129/5128 [==============================] - 891s 174ms/step - loss: 209.3634 - mse: 209.3724 - val_loss: 151.1521 - val_mse: 151.0975
Epoch 2/10
5129/5128 [==============================] - 838s 163ms/step - loss: 130.8876 - mse: 130.8973 - val_loss: 123.0750 - val_mse: 123.0214
Epoch 3/10
5129/5128 [==============================] - 824s 161ms/step - loss: 112.3384 - mse: 112.3301 - val_loss: 122.3660 - val_mse: 122.3201
Epoch 4/10
5129/5128 [==============================] - 831s 162ms/step - loss: 99.4011 - mse: 99.3963 - val_loss: 123.7437 - val_mse: 123.7186
Epoch 5/10
5129/5128 [==============================] - 828s 161ms/step - loss: 88.0731 - mse: 88.0815 - val_loss: 106.6786 - val_mse: 106.6540
Epoch 6/10
5129/5128 [==============================] - 826s 161ms/step - loss: 78.6207 - mse: 78.6049 - val_loss: 108.5776 - val_mse: 108.5595
Epoch 7/10
5129/5128 [==============================] - 826s 161ms/step - loss: 68.9992 - mse: 69.0038 - val_loss: 106.2708 - val_

In [24]:
print(history.history)

{'loss': [209.37247939356348, 130.89733179491705, 112.329916109771, 99.39633042176366, 88.0813424002176, 78.60487301518678, 69.00396133726902, 60.37362299094757, 52.752996699251625, 46.7506385540084], 'mse': [209.37242, 130.89731, 112.330055, 99.396286, 88.08154, 78.60493, 69.0038, 60.37367, 52.753036, 46.75062], 'val_loss': [151.15205725899676, 123.07500939862409, 122.36595800679491, 123.74365729779362, 106.67858649728015, 108.57758578582593, 106.27078395022156, 110.82191314144957, 111.8396373454633, 113.7464084991358], 'val_mse': [151.0975, 123.02143, 122.320114, 123.71861, 106.65402, 108.55953, 106.223885, 110.8106, 111.75982, 113.71804], 'lr': [1e-04, 1e-04, 1e-04, 1e-04, 1e-04, 1e-04, 1e-04, 1e-04, 1e-04, 1e-04]}


In [25]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training accuracy')
plt.plot(val_acc, label='Validation accuracy')
plt.legend(loc='lower right')
plt.ylabel('accuracy')
plt.ylim([min(plt.ylim()),500])
plt.title('Training and Validation accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
plt.ylim([0,20000])
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()


# In[ ]:


print(history.history)

KeyError: 'accuracy'

In [ ]:
test_datagen = ImageDataGenerator(rescale=1./255)


test_generator = test_datagen.flow_from_dataframe(dataframe=test_df,
                                    directory=test_set,
                                    x_col="image_id",
                                    y_col=["x_1","y_1","width","height"],
                                    class_mode="raw",
                                    shuffle=True,
                                    target_size=(224, 224),
                                    batch_size=BATCH_SIZE)

STEP_SIZE_TEST = test_generator.n / test_generator.batch_size 


#CHANGE PARALLEL MODEL
pred=model.predict_generator(test_generator,  steps=STEP_SIZE_TEST,  verbose=1)


In [ ]:
print(pred[1])

In [ ]:
from PIL import  Image, ImageDraw

count = 0

for gen_image in test_generator:
    img = Image.fromarray(np.uint8((gen_image[0][0])*255))
    img1 = ImageDraw.Draw(img)
    x1,y1 = pred[count][0],pred[count][1]
    x4,y4= pred[count][0] + pred[count][2], pred[count][1] + pred[count][3]
    img1.rectangle([(x1,y1),(x4,y4)], outline ="red") 
    #img.show() 
    img.save("/home/lorenzo.stacchio/ML/Machine_learning_project/dataset/NUOVO/results/output"+ str(count) + ".jpg")
    #print(gen_image[0][0].shape,pred[count])
    count = count + 1
    if count == 1000:
        break